## Блок импорта ##

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

## Блок загрузки данных ##

Для примера пусть будет классификация ирисов Фишера.

In [2]:
df = load_iris()

In [3]:
X, y = pd.DataFrame(df['data']), df['target']

In [4]:
X.shape

(150, 4)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle = True)

In [6]:
np.unique(y_test), np.unique(y_train)

(array([0, 1, 2]), array([0, 1, 2]))

## Сам классификатор KNN ##

In [7]:
class KNNClassifier():
    def __init__(self, n_neighbours = 3):
        self.n_neighbours = n_neighbours
        
    def mse(self, a, b):
        a = np.array(a)
        b = np.array(b)
        return (sum((a - b)**2))**.5
    
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def voting(self, best_neighbours_for_every_point):
        """На вход - массив (X_new.shape[0] * n_neighbours * 2), т.к. 2 - это [dist, label] 
        Возвращает - массив (X_new.shape[0] * 1 ), 1 - это метки """
        y_pred = np.array([np.median( np.array(test_sample)[:,1] ) for test_sample in best_neighbours_for_every_point] )
        # test_sample имеет форму (n,2) для n соседей
        return y_pred
    
    def predict(self, X_new):
        dist = np.zeros((X_new.shape[0], self.X.shape[0]))
        # для каждой точки
        for idx_point, new_point in enumerate(X_new):
            # посчитать расстояния до каждой из других точек
            for idx_neighbor, old_neighbor in enumerate(self.X):
                dist[idx_point, idx_neighbor] = self.mse(new_point, old_neighbor)
                
        # dist - матрица n*m, где n - новые точки, m - старые точки
        # и dist[n,m] - расстояние от новой точки до старой (из трейна)
        #print(dist.shape) # 50*100
        # print(self.y.shape) по итогу есть 50 новых точек и 100 расстояний для каждой до всех точек трейна
        #print(dist)
        #print(self.y)
        # теперь надо просто отсортировать расстояния для каждой точки и применить к ней метку соответствующего мажоритарного голосования
        
        # продублировать n раз игреки для соотнесения
        Y = np.array([list(self.y)] * X_new.shape[0])
        # print(np.array(Y).shape)
        #print(dist, Y)
        df = [[(x,y) for x,y in zip(dist_to_old_points, labels_of_old_points)] for dist_to_old_points, labels_of_old_points in zip(dist, Y)]
        df = np.array(df)#.reshape(2,50,100)
        
        # теперь нужно отсортировать соседей и взять n - ближайших
        #print(df)
        neighbours = np.array([sorted(x_50, key = lambda x: x[0])[:self.n_neighbours] for x_50 in df])
        #print(neighbours)
        #print(neighbours.shape) # (50, 5, 2) - 50 точек, для каждой их них 5 ближайших
        # получили n ближайших соседей, теперь для каждой точки нужно выработать предсказание
        y_pred = self.voting(neighbours)
        return y_pred

## Тестирование ##

In [8]:
knn = KNNClassifier(3)
knn.fit(X_train.to_numpy(), y_train)
y_pred = knn.predict(X_test.to_numpy())
y_pred

array([2., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 2., 0., 1., 1., 0., 2.,
       1., 0., 1., 1., 2., 2., 1., 1., 0., 2., 1., 0., 1., 2., 0., 1., 1.,
       1., 0., 0., 0.])

In [9]:
[(i,j) for i,j in zip(y_pred.astype('int32'), y_test)] # вывод в кортежах (y_pred, y_true)

[(2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 2),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (2, 2),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (2, 2),
 (2, 2),
 (1, 1),
 (1, 1),
 (0, 0),
 (2, 2),
 (1, 1),
 (0, 0),
 (1, 1),
 (2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0)]

In [10]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        16
         1.0       1.00      0.93      0.97        15
         2.0       0.88      1.00      0.93         7

    accuracy                           0.97        38
   macro avg       0.96      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## Сравнение с sklearn-новским ##

In [11]:
k = KNeighborsClassifier(n_neighbors = 3)
k.fit(X_train.to_numpy(), y_train)
y_pred_skl = knn.predict(X_test.to_numpy())
y_pred_skl

array([2., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 2., 0., 1., 1., 0., 2.,
       1., 0., 1., 1., 2., 2., 1., 1., 0., 2., 1., 0., 1., 2., 0., 1., 1.,
       1., 0., 0., 0.])

In [12]:
[(i,j) for i,j in zip(y_pred_skl.astype('int32'), y_test)] # вывод в кортежах (y_pred, y_true)

[(2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (1, 2),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (0, 0),
 (2, 2),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 1),
 (2, 2),
 (2, 2),
 (1, 1),
 (1, 1),
 (0, 0),
 (2, 2),
 (1, 1),
 (0, 0),
 (1, 1),
 (2, 2),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 0)]

In [13]:
print(classification_report(y_pred_skl, y_test))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        16
         1.0       1.00      0.93      0.97        15
         2.0       0.88      1.00      0.93         7

    accuracy                           0.97        38
   macro avg       0.96      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



**Вывод: Получилось такая же метрика, все отлично работает.**